In [1]:
import ngsolve as ngs
from ngsolve.webgui import Draw
import numpy as np

mesh = ngs.Mesh('proximity_geo.vol')
mesh.Curve(2)
# mesh.RefineHP(3,factor = 0.2)
mesh.Refine()

In [2]:
# Parameters

circ_mass = np.pi*0.031**2
circ_number = 58

mu0 = 1.256*1e-6
mu_iron = 1000*mu0
sigma = 58*1e6
J = 1e7

circles_left = ""; 
for i in range(58): circles_left+="circle_left_"+str(i)+"|"
circles_right = ""; 
for i in range(58): circles_right+="circle_right_"+str(i)+"|"

cf_Js = mesh.MaterialCF({circles_left: J, circles_right: -J}, default = 0)
cf_mu = mesh.MaterialCF({ circles_left +circles_right +"left|right|bridge" : mu0, "ambient": mu_iron})
cf_sigma = mesh.MaterialCF({circles_left +circles_right: sigma}, default = 0)

f = 0 #(Hz)
w = f*2*np.pi

# H1 = ngs.H1(mesh, order = 2, dirichlet = "ambient_face")
H1 = ngs.H1(mesh, order = 3, dirichlet = "ambient_face", complex = True)
u,v = H1.TnT()

K = ngs.BilinearForm(H1)
K += (1/cf_mu*ngs.grad(u)*ngs.grad(v))*ngs.dx + ((1j*w)*sigma*u*v)*ngs.dx(circles_left +circles_right)
K.Assemble()

f = ngs.LinearForm(H1)
f += (cf_Js*v)*ngs.dx()
f.Assemble()

A = ngs.GridFunction(H1)
A.vec.data += K.mat.Inverse(H1.FreeDofs())*f.vec

B = ngs.grad(A)

J = (cf_Js -(1j*w)*cf_sigma*A)

# J = J/intJ*I

In [3]:
# Draw(ngs.Norm(J), mesh, settings = {"Objects": {"Wireframe": False}})
# Draw(ngs.Norm(B),mesh, settings = {"Objects": {"Wireframe": False}})

# Draw(ngs.log(ngs.Norm(J)), mesh, settings = {"Objects": {"Wireframe": False}})
# Draw(ngs.log(ngs.Norm(B)), mesh, settings = {"Objects": {"Wireframe": False}})
Draw((J), mesh, settings = {"Objects": {"Wireframe": False}})

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {'Objects': {'Wireframe': Fal…

BaseWebGuiScene

In [4]:
ngs.Integrate(1/sigma*ngs.Norm(J)*ngs.Norm(J), mesh)

59.717485021790026